## Model 4

In [ ]:
# Fitting Kernel SVM to the Training set
from sklearn.svm import SVC
ksvm = SVC(kernel = 'poly', random_state = 0)
#‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’

ksvm_bow = ksvm.fit(cv_train_text, train_sentiments)
# Predicting the Test set results
ksvm_bow_predict = ksvm.predict(cv_test_text)

ksvm_tfidf = ksvm.fit(tv_train_text, train_sentiments)
# Predicting the Test set results
ksvm_tfidf_predict = ksvm.predict(tv_test_text)

#confusion matrix for tfidf features
cm_tfidf=confusion_matrix(test_sentiments,ksvm_tfidf_predict,labels=[1,0])
conf_matrices["ksvm_tfidf"] = cm_tfidf

plt.figure(figsize=(10,7))
sns.set(font_scale=1.4)  # for label size
sns.heatmap(cm_tfidf, annot=True, annot_kws={"size": 16}, fmt='g')  # fmt='g' is a format option to suppress scientific notation

plt.ylabel('Actual label')
plt.xlabel('Predicted label')
plt.title('TF-IDF Confusion Matrix Heatmap')
plt.show()

#confusion matrix for bag of words
cm_bow=confusion_matrix(test_sentiments,ksvm_bow_predict,labels=[1,0])
conf_matrices["ksvm_bow"] = cm_bow

plt.figure(figsize=(10,7))
sns.set(font_scale=1.4)  # for label size
sns.heatmap(cm_bow, annot=True, annot_kws={"size": 16}, fmt='g')  # fmt='g' is a format option to suppress scientific notation

plt.ylabel('Actual label')
plt.xlabel('Predicted label')
plt.title('BoW Confusion Matrix Heatmap')
plt.show()

## Model 5

In [ ]:
# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])

from sklearn.compose import ColumnTransformer
ct = ColumnTransformer([("X", OneHotEncoder(dtype = int), [1])], remainder = 'passthrough')
X = ct.fit_transform(X)
X = X[:, 1:]

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


In [ ]:
# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense

# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 24, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))

# Adding the second hidden layer
classifier.add(Dense(units = 12, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the third hidden layer
classifier.add(Dense(units = 12, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the fourth hidden layer
classifier.add(Dense(units = 12, kernel_initializer = 'uniform', activation = 'relu'))


# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
classifier.fit(cv_train_text, train_sentiments, batch_size = 20, epochs = 500)


In [ ]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [ ]:
# Making the Confusion Matrix
cm_tfidf = confusion_matrix(y_test, y_pred)
conf_matrices["ann_tfidf"] = cm_tfidf

# Making the Confusion Matrix
cm_bow = confusion_matrix(y_test, y_pred)
conf_matrices["ann_bow"] = cm_bow

# Conclusion

After executing these four models, we can see from the confusions matrixes that in every case we have fewer False Positives by using BoW than TFIDF so we will only work with BoW from this point onwards.

We can also see that Kernel SVM has the highest amount of FP, so we will only work with the other 3 models to tune their hyperparameters and build better models. 